In [1]:
import pandas as pd
import numpy as np
import datetime
import random
import string
import joblib # For loading models
import matplotlib.pyplot as plt
import seaborn as sns

# --- Configuration and Global Variables (consistent with previous steps) ---
NUM_CUSTOMERS = 500
NUM_TRANSACTIONS = 100000

# Country Risk Map (simulated external data)
COUNTRY_RISK_MAP = {
    'IRAN': 'HIGH', 'NORTH KOREA': 'HIGH', 'SYRIA': 'HIGH', 'CUBA': 'HIGH', 'VENEZUULA': 'HIGH',
    'RUSSIA': 'MEDIUM', 'CHINA': 'MEDIUM', 'INDIA': 'LOW', 'USA': 'LOW', 'UK': 'LOW',
    'GERMANY': 'LOW', 'FRANCE': 'LOW', 'BRAZIL': 'MEDIUM', 'SOUTH AFRICA': 'MEDIUM',
    'NIGERIA': 'MEDIUM', 'AFGHANISTAN': 'HIGH', 'YEMEN': 'HIGH', 'SOMALIA': 'HIGH',
    'LEBANON': 'MEDIUM', 'PAKISTAN': 'MEDIUM'
}
HIGH_RISK_COUNTRIES = [country for country, risk in COUNTRY_RISK_MAP.items() if risk == 'HIGH']
LOW_RISK_COUNTRIES = [country for country, risk in COUNTRY_RISK_MAP.items() if risk == 'LOW']

# --- Helper Functions for Data Loading and Generation (copied for self-containment) ---

def load_common_names_from_excel(filepath, sheet_name):
    """
    Loads a list of full names from an Excel file.
    Assumes the names are in a column named 'Sanctioned_name' in the specified sheet.
    Provides a fallback to a hardcoded list if the file or column is not found.
    """
    try:
        names_df = pd.read_excel(filepath, sheet_name=sheet_name)
        if 'Sanctioned_name' in names_df.columns:
            return names_df['Sanctioned_name'].astype(str).tolist()
        else:
            return _get_default_common_names()
    except FileNotFoundError:
        return _get_default_common_names()
    except Exception as e:
        return _get_default_common_names()

def _get_default_common_names():
    """Provides a hardcoded list of common names as a fallback."""
    return [
        'John Smith', 'Jane Johnson', 'Michael Williams', 'Emily Brown', 'David Jones',
        'Sarah Garcia', 'Chris Miller', 'Anna Davis', 'Robert Rodriguez', 'Maria Martinez',
        'William Taylor', 'Olivia Wilson', 'James Moore', 'Sophia White', 'Benjamin Green',
        'Isabella Hall', 'Lucas King', 'Mia Wright', 'Henry Lopez', 'Charlotte Hill'
    ]

def load_or_generate_initial_data(sanctions_csv_path='UK Sanctions List_mean.csv',
                                   customer_data_path='customer_data.csv',
                                   num_customers=None):
    """
    Loads cleaned sanctions and customer data. If files are not found,
    it generates minimal dummy data for demonstration.
    """
    if num_customers is None:
        num_customers = NUM_CUSTOMERS

    global COMMON_FULL_NAMES
    COMMON_FULL_NAMES = load_common_names_from_excel('Name_list.xlsx', 'Names')

    if not COMMON_FULL_NAMES:
        print("FATAL: COMMON_FULL_NAMES list is empty. Cannot proceed with data generation.")
        return pd.DataFrame(), pd.DataFrame()

    sanctions_df_cleaned = pd.DataFrame()
    customer_df = pd.DataFrame()

    try:
        raw_sanctions_df = pd.read_csv(sanctions_csv_path, encoding='latin1', header=1)
        print(f"Successfully loaded {sanctions_csv_path} with 'latin1' encoding. Shape: {raw_sanctions_df.shape}")

        name_col = 'Name 6'
        address_col = 'Address 6'
        dob_col = 'DOB 6'
        nationality_col = 'Nationality 6'
        type_col = 'Type'
        id_col = 'ID'

        actual_name_col = name_col if name_col in raw_sanctions_df.columns else ('Name' if 'Name' in raw_sanctions_df.columns else None)
        actual_address_col = address_col if address_col in raw_sanctions_df.columns else ('Address' if 'Address' in raw_sanctions_df.columns else None)
        actual_dob_col = dob_col if dob_col in raw_sanctions_df.columns else ('DOB' if 'DOB' in raw_sanctions_df.columns else None)
        actual_nationality_col = nationality_col if nationality_col in raw_sanctions_df.columns else ('Nationality' if 'Nationality' in raw_sanctions_df.columns else None)
        actual_type_col = type_col if type_col in raw_sanctions_df.columns else ('Type' if 'Type' in raw_sanctions_df.columns else None)
        actual_id_col = id_col if id_col in raw_sanctions_df.columns else ('ID' if 'ID' in raw_sanctions_df.columns else None)

        if not actual_name_col:
            print(f"FATAL: No suitable name column found in sanctions CSV. Using dummy data for sanctions.")
            raise ValueError("No name column found")

        sanctions_df_cleaned = raw_sanctions_df.copy()
        sanctions_df_cleaned['Sanctioned_Name'] = sanctions_df_cleaned[actual_name_col].astype(str).str.upper().str.strip()
        
        sanctions_df_cleaned['Sanctioned_Address'] = sanctions_df_cleaned[actual_address_col].astype(str).str.upper().str.strip() if actual_address_col and actual_address_col in sanctions_df_cleaned.columns else np.nan
        sanctions_df_cleaned['Sanctioned_DOB'] = pd.to_datetime(sanctions_df_cleaned[actual_dob_col], errors='coerce').dt.strftime('%Y-%m-%d') if actual_dob_col and actual_dob_col in sanctions_df_cleaned.columns else np.nan
        sanctions_df_cleaned['Sanctioned_Nationality'] = sanctions_df_cleaned[actual_nationality_col].astype(str).str.upper().str.strip() if actual_nationality_col and actual_nationality_col in sanctions_df_cleaned.columns else np.nan
        sanctions_df_cleaned['Sanction_Type'] = sanctions_df_cleaned[actual_type_col].astype(str).str.upper().str.strip() if actual_type_col and actual_type_col in sanctions_df_cleaned.columns else np.nan
        sanctions_df_cleaned['Sanctioned_ID'] = sanctions_df_cleaned[actual_id_col].astype(str) if actual_id_col and actual_id_col in sanctions_df_cleaned.columns else [f'S{i:04d}' for i in range(len(sanctions_df_cleaned))]

        sanctions_df_cleaned = sanctions_df_cleaned[[
            'Sanctioned_ID', 'Sanctioned_Name', 'Sanctioned_Address',
            'Sanctioned_DOB', 'Sanctioned_Nationality', 'Sanction_Type'
        ]].copy()
        sanctions_df_cleaned = sanctions_df_cleaned[
            (sanctions_df_cleaned['Sanctioned_Name'] != 'UNKNOWN SANCTIONED NAME') &
            (sanctions_df_cleaned['Sanctioned_Name'] != 'NAN') &
            (sanctions_df_cleaned['Sanctioned_Name'].str.strip() != '')
        ].reset_index(drop=True)
        print("Cleaned Sanctions Data (first 3 rows):")
        print(sanctions_df_cleaned.head(3))
        sanctions_df_cleaned.to_csv('sanctions_list_cleaned.csv', index=False)

    except FileNotFoundError:
        print(f"Error: Sanctions file '{sanctions_csv_path}' not found. Generating minimal dummy sanctions data.")
        sanctions_df_cleaned = pd.DataFrame({
            'Sanctioned_ID': [f'S{i:04d}' for i in range(1, 101)],
            'Sanctioned_Name': [f'SANCTIONED PERSON {i}' for i in range(1, 101)],
            'Sanctioned_Address': [f'{i*10} MAIN ST, HIGH RISK COUNTRY' for i in range(1, 101)],
            'Sanctioned_DOB': [f'{1950 + i}-01-01' for i in range(100)],
            'Sanctioned_Nationality': random.choices(HIGH_RISK_COUNTRIES, k=100),
            'Sanction_Type': random.choices(['INDIVIDUAL', 'ENTITY'], k=100)
        })
        print("Generated fallback sanctions data.")
    except Exception as e:
        print(f"An unexpected error occurred during sanctions data loading/cleaning: {e}. Generating dummy sanctions data.")
        sanctions_df_cleaned = pd.DataFrame({
            'Sanctioned_ID': [f'S{i:04d}' for i in range(1, 101)],
            'Sanctioned_Name': [f'SANCTIONED PERSON {i}' for i in range(1, 101)],
            'Sanctioned_Address': [f'{i*10} MAIN ST, HIGH RISK COUNTRY' for i in range(1, 101)],
            'Sanctioned_DOB': [f'{1950 + i}-01-01' for i in range(100)],
            'Sanctioned_Nationality': random.choices(HIGH_RISK_COUNTRIES, k=100),
            'Sanction_Type': random.choices(['INDIVIDUAL', 'ENTITY'], k=100)
        })
        print("Generated fallback sanctions data due to error.")


    try:
        customer_df = pd.read_csv(customer_data_path)
        print(f"Successfully loaded customer data from {customer_data_path}. Shape: {customer_df.shape}")

        current_cols_lower = {col.lower(): col for col in customer_df.columns}
        
        expected_customer_cols_mapping = {
            'customer_id': 'Customer_ID',
            'customer_name': 'Customer_Name',
            'customer_address': 'Customer_Address',
            'customer_dob': 'Customer_DOB',
            'customer_nationality': 'Customer_Nationality',
            'customer_country': 'Customer_Country',
            'customer_industry': 'Customer_Industry',
            'onboarding_date': 'Onboarding_Date'
        }
        
        rename_dict = {}
        for old_col_lower, new_col_proper in expected_customer_cols_mapping.items():
            if old_col_lower in current_cols_lower:
                rename_dict[current_cols_lower[old_col_lower]] = new_col_proper
            elif new_col_proper not in customer_df.columns:
                print(f"Warning: Customer column '{new_col_proper}' not found in loaded data. Creating as NaN.")
                customer_df[new_col_proper] = np.nan

        if rename_dict:
            customer_df.rename(columns=rename_dict, inplace=True)
            print(f"Standardized customer column names: {rename_dict}")
            
        print(f"Customer DataFrame columns after standardization: {customer_df.columns.tolist()}")

        required_customer_cols = ['Customer_ID', 'Customer_Name', 'Customer_Address', 'Customer_DOB', 'Customer_Nationality', 'Customer_Country']
        if not all(col in customer_df.columns for col in required_customer_cols):
            missing_cols = [col for col in required_customer_cols if col not in customer_df.columns]
            print(f"FATAL: Missing required customer columns after loading/standardization: {missing_cols}. Cannot proceed.")
            customer_df = pd.DataFrame() 

    except FileNotFoundError:
        print(f"Customer data file '{customer_data_path}' not found. Generating dummy customer data.")
        customers = []
        for i in range(1, num_customers + 1):
            customer_id = f'CUST{i:05d}'
            customer_name = random.choice(COMMON_FULL_NAMES)
            customer_address = f"{random.randint(100, 999)} {random.choice(['Main St', 'Oak Ave', 'Pine Ln'])}"
            customer_dob = (datetime.date(1950, 1, 1) + datetime.timedelta(days=random.randint(0, 365 * 50))).strftime('%Y-%m-%d')
            customer_nationality = random.choice(list(COUNTRY_RISK_MAP.keys()))
            customer_country = random.choice(list(COUNTRY_RISK_MAP.keys()))
            customer_industry = random.choice(['Financial Services', 'Retail', 'Technology', 'Manufacturing', 'Healthcare'])
            onboarding_date = (datetime.date(2020, 1, 1) + datetime.timedelta(days=random.randint(0, 365 * 3))).strftime('%Y-%m-%d')

            if i % 10 == 0 and not sanctions_df_cleaned.empty:
                sanctioned_entity = sanctions_df_cleaned.sample(1).iloc[0]
                customer_name = sanctioned_entity['Sanctioned_Name'].replace('A', 'a', 1).replace('E', 'e', 1)
                customer_address = sanctioned_entity['Sanctioned_Address'].replace('ST', 'Street', 1)
                customer_dob = sanctioned_entity['Sanctioned_DOB']
                customer_nationality = sanctioned_entity['Sanctioned_Nationality']
                customer_country = sanctioned_entity['Sanctioned_Nationality']

            customers.append({
                'Customer_ID': customer_id,
                'Customer_Name': customer_name,
                'Customer_Address': customer_address,
                'Customer_DOB': customer_dob,
                'Customer_Nationality': customer_nationality,
                'Customer_Country': customer_country,
                'Customer_Industry': customer_industry,
                'Onboarding_Date': onboarding_date
            })
        customer_df = pd.DataFrame(customers)
        customer_df.to_csv(customer_data_path, index=False)
        print("Generated dummy customer data.")
    except Exception as e:
        print(f"An unexpected error occurred during customer data loading/generation: {e}. Generating dummy customer data.")
        customers = []
        for i in range(1, num_customers + 1):
            customer_id = f'CUST{i:05d}'
            customer_name = random.choice(COMMON_FULL_NAMES)
            customer_address = f"{random.randint(100, 999)} {random.choice(['Main St', 'Oak Ave', 'Pine Ln'])}"
            customer_dob = (datetime.date(1950, 1, 1) + datetime.timedelta(days=random.randint(0, 365 * 50))).strftime('%Y-%m-%d')
            customer_nationality = random.choice(list(COUNTRY_RISK_MAP.keys()))
            customer_country = random.choice(list(COUNTRY_RISK_MAP.keys()))
            customer_industry = random.choice(['Financial Services', 'Retail', 'Technology', 'Manufacturing', 'Healthcare'])
            onboarding_date = (datetime.date(2020, 1, 1) + datetime.timedelta(days=random.randint(0, 365 * 3))).strftime('%Y-%m-%d')
            customers.append({
                'Customer_ID': customer_id, 'Customer_Name': customer_name, 'Customer_Address': customer_address,
                'Customer_DOB': customer_dob, 'Customer_Nationality': customer_nationality, 'Customer_Country': customer_country,
                'Customer_Industry': customer_industry, 'Onboarding_Date': onboarding_date
            })
        customer_df = pd.DataFrame(customers)
        print("Generated fallback customer data due to error.")

    return sanctions_df_cleaned, customer_df

# --- New Functions for Integrated Risk Scoring and Alert Management (Step 5) ---

def calculate_integrated_risk(customer_df, sanctions_results_df, aml_results_df):
    """
    Combines results from sanctions screening and AML transaction monitoring
    to calculate an integrated risk score for each customer.
    """
    print("\n--- Calculating Integrated Risk Scores ---")

    # Ensure customer_df has a 'Customer_Country' column for risk mapping
    if 'Customer_Country' not in customer_df.columns:
        print("Warning: 'Customer_Country' not found in customer_df. Adding as NaN and mapping to 0 risk.")
        customer_df['Customer_Country'] = np.nan
    
    # Map customer country risk (from previous step's logic)
    customer_df['Customer_Country_Risk_Score'] = customer_df['Customer_Country'].astype(str).str.upper().map(
        {k: (10 if v == 'HIGH' else 5 if v == 'MEDIUM' else 1) for k, v in COUNTRY_RISK_MAP.items()}
    ).fillna(0)

    # Merge sanctions results
    # Use 'how=left' to keep all customers, even if they had no sanctions alerts
    customer_risk_df = pd.merge(
        customer_df[['Customer_ID', 'Customer_Name', 'Customer_Country_Risk_Score']],
        sanctions_results_df[['Customer_ID', 'Max_Sanction_Match_Probability', 'Sanction_Alert_Flag']],
        on='Customer_ID',
        how='left'
    )
    # Fill NaNs for customers with no sanctions alerts
    customer_risk_df['Max_Sanction_Match_Probability'] = customer_risk_df['Max_Sanction_Match_Probability'].fillna(0)
    customer_risk_df['Sanction_Alert_Flag'] = customer_risk_df['Sanction_Alert_Flag'].fillna('OK')

    # Merge AML results (aggregate AML alerts per customer)
    # For AML, we're interested in the *highest* anomaly score (lowest decision_function) for a customer
    # and the count of alerts.
    aml_customer_summary = aml_results_df[aml_results_df['AML_Alert_Flag'] == 'ALERT'].groupby('Customer_ID').agg(
        Num_AML_Alerts=('Transaction_ID', 'count'),
        Min_Anomaly_Score=('Anomaly_Score', 'min') # Lower score means higher anomaly
    ).reset_index()

    # Merge AML summary into the main customer risk DataFrame
    customer_risk_df = pd.merge(
        customer_risk_df,
        aml_customer_summary,
        on='Customer_ID',
        how='left'
    )
    # Fill NaNs for customers with no AML alerts
    customer_risk_df['Num_AML_Alerts'] = customer_risk_df['Num_AML_Alerts'].fillna(0)
    customer_risk_df['Min_Anomaly_Score'] = customer_risk_df['Min_Anomaly_Score'].fillna(1) # Max Isolation Forest score is 1

    # --- Calculate Integrated Risk Score ---
    # This is a simplified weighted sum. Weights can be tuned based on business rules and risk appetite.
    # Higher score = higher risk.
    # Sanctions probability is already 0-100.
    # AML anomaly score is typically negative for anomalies, positive for normal.
    # We'll normalize AML anomaly score to contribute positively to risk.
    
    # Normalize AML anomaly score: (1 - score) makes lower scores (more anomalous) higher risk.
    # We'll clip it to ensure it's within a reasonable range, e.g., 0 to 1.
    # Max possible anomaly score is around 0.5 for IsolationForest, min is -0.5 to -1.
    # Let's map it to a 0-100 scale where 100 is most anomalous.
    # A simple way is to take (max_score - actual_score) / (max_score - min_score)
    # For Isolation Forest, decision_function usually ranges from approx -0.5 to 0.5.
    # Let's map -0.5 to 100, 0.5 to 0.
    
    # Example mapping:
    # If Min_Anomaly_Score is -0.5 (very anomalous), mapped_aml_risk = 100
    # If Min_Anomaly_Score is 0.5 (very normal), mapped_aml_risk = 0
    
    # Simple linear mapping for illustration:
    min_if_score = customer_risk_df['Min_Anomaly_Score'].min()
    max_if_score = customer_risk_df['Min_Anomaly_Score'].max()
    
    # Avoid division by zero if all scores are the same
    if max_if_score - min_if_score == 0:
        customer_risk_df['Mapped_AML_Risk'] = 0 # No variation, no risk from this component
    else:
        customer_risk_df['Mapped_AML_Risk'] = (max_if_score - customer_risk_df['Min_Anomaly_Score']) / (max_if_score - min_if_score) * 100
    
    # Ensure it's not negative and cap at 100
    customer_risk_df['Mapped_AML_Risk'] = customer_risk_df['Mapped_AML_Risk'].clip(lower=0, upper=100)
    
    # Define weights for each component
    W_CUSTOMER_RISK = 0.3 # Weight for inherent customer risk (e.g., country)
    W_SANCTIONS_ALERT = 0.4 # Weight for sanctions match probability
    W_AML_ALERTS = 0.3 # Weight for transaction anomaly score/count

    customer_risk_df['Integrated_Risk_Score'] = (
        customer_risk_df['Customer_Country_Risk_Score'] * W_CUSTOMER_RISK +
        customer_risk_df['Max_Sanction_Match_Probability'] * W_SANCTIONS_ALERT +
        customer_risk_df['Mapped_AML_Risk'] * W_AML_ALERTS
    )

    # Normalize Integrated_Risk_Score to a 0-100 scale (optional, but good for interpretability)
    max_possible_integrated_score = (10 * W_CUSTOMER_RISK + 100 * W_SANCTIONS_ALERT + 100 * W_AML_ALERTS)
    customer_risk_df['Integrated_Risk_Score'] = (customer_risk_df['Integrated_Risk_Score'] / max_possible_integrated_score) * 100
    
    # Classify overall risk level
    customer_risk_df['Overall_Risk_Level'] = pd.cut(
        customer_risk_df['Integrated_Risk_Score'],
        bins=[0, 30, 60, 100],
        labels=['LOW', 'MEDIUM', 'HIGH'],
        right=False,
        include_lowest=True
    )

    print("\nIntegrated Risk Score Calculation Complete. Sample of results:")
    print(customer_risk_df[['Customer_ID', 'Customer_Name', 'Customer_Country_Risk_Score',
                            'Max_Sanction_Match_Probability', 'Num_AML_Alerts', 'Min_Anomaly_Score',
                            'Integrated_Risk_Score', 'Overall_Risk_Level']].sort_values(by='Integrated_Risk_Score', ascending=False).head())
    
    return customer_risk_df

def generate_alerts_summary(integrated_risk_df, alert_threshold_score=50):
    """
    Generates a summary of alerts based on the integrated risk score.
    """
    print(f"\n--- Generating Consolidated Alerts Summary (Threshold: {alert_threshold_score}) ---")

    alerts_df = integrated_risk_df[integrated_risk_df['Integrated_Risk_Score'] >= alert_threshold_score].copy()
    
    if alerts_df.empty:
        print("No high-risk alerts generated based on the integrated risk score threshold.")
        return pd.DataFrame(columns=['Customer_ID', 'Customer_Name', 'Integrated_Risk_Score', 'Overall_Risk_Level',
                                     'Sanction_Alert_Flag', 'Num_AML_Alerts', 'Alert_Reason'])

    # Add a simple 'Alert_Reason' column
    alerts_df['Alert_Reason'] = ''
    alerts_df.loc[alerts_df['Sanction_Alert_Flag'] == 'ALERT', 'Alert_Reason'] += 'Sanctions Match; '
    alerts_df.loc[alerts_df['Num_AML_Alerts'] > 0, 'Alert_Reason'] += 'Transaction Anomaly; '
    alerts_df.loc[alerts_df['Customer_Country_Risk_Score'] >= 5, 'Alert_Reason'] += 'High Country Risk; '
    alerts_df['Alert_Reason'] = alerts_df['Alert_Reason'].str.strip('; ')
    alerts_df.loc[alerts_df['Alert_Reason'] == '', 'Alert_Reason'] = 'High Integrated Risk' # Fallback

    # Sort alerts by risk score
    alerts_df = alerts_df.sort_values(by='Integrated_Risk_Score', ascending=False).reset_index(drop=True)

    print(f"Total High-Risk Alerts: {len(alerts_df)}")
    print("\nTop 10 Consolidated Alerts:")
    print(alerts_df[['Customer_ID', 'Customer_Name', 'Integrated_Risk_Score', 'Overall_Risk_Level', 'Alert_Reason']].head(10))

    # Visualize alert distribution
    plt.figure(figsize=(8, 6))
    sns.countplot(x='Overall_Risk_Level', data=alerts_df, order=['LOW', 'MEDIUM', 'HIGH'], palette='viridis')
    plt.title('Distribution of Overall Risk Levels for Alerts')
    plt.xlabel('Risk Level')
    plt.ylabel('Number of Customers')
    plt.show()

    return alerts_df[['Customer_ID', 'Customer_Name', 'Integrated_Risk_Score', 'Overall_Risk_Level', 'Sanction_Alert_Flag', 'Num_AML_Alerts', 'Alert_Reason']]


# --- Main Execution Flow for Integrated Risk Scoring and Alert Management ---
if __name__ == "__main__":
    print("--- Starting Integrated Risk Scoring and Alert Management ---")

    # 1. Load Initial Customer Data
    # We need the full customer_df for customer country risk and merging
    sanctions_dummy_df, customer_df = load_or_generate_initial_data()
    if customer_df.empty:
        print("FATAL: Customer data could not be loaded or generated. Exiting.")
        exit()

    # 2. Load Results from Previous ML Steps
    try:
        sanctions_results_df = pd.read_csv('final_sanctions_screening_results.csv')
        print(f"Loaded Sanctions Screening Results. Shape: {sanctions_results_df.shape}")
    except FileNotFoundError:
        print("Warning: 'final_sanctions_screening_results.csv' not found. Creating dummy sanctions results.")
        # Create a dummy sanctions results DataFrame if the file is missing
        sanctions_results_df = customer_df[['Customer_ID', 'Customer_Name']].copy()
        sanctions_results_df['Max_Sanction_Match_Probability'] = 0
        sanctions_results_df['Sanction_Alert_Flag'] = 'OK'
        # Inject a few dummy alerts for demonstration
        if not sanctions_results_df.empty:
            num_dummy_alerts = min(10, len(sanctions_results_df) // 10)
            dummy_alert_indices = random.sample(range(len(sanctions_results_df)), num_dummy_alerts)
            sanctions_results_df.loc[dummy_alert_indices, 'Max_Sanction_Match_Probability'] = np.random.uniform(0.6, 0.9, num_dummy_alerts) * 100
            sanctions_results_df.loc[dummy_alert_indices, 'Sanction_Alert_Flag'] = 'ALERT'


    try:
        aml_results_df = pd.read_csv('final_aml_screening_results.csv')
        print(f"Loaded AML Transaction Monitoring Results. Shape: {aml_results_df.shape}")
    except FileNotFoundError:
        print("Warning: 'final_aml_screening_results.csv' not found. Creating dummy AML results.")
        # Create a dummy AML results DataFrame if the file is missing
        aml_results_df = pd.DataFrame(columns=['Transaction_ID', 'Customer_ID', 'Amount_USD', 'Anomaly_Score', 'AML_Alert_Flag', 'Is_Suspicious_Label'])
        # Generate some dummy AML alerts if customer_df is available
        if not customer_df.empty:
            dummy_transactions = []
            customer_ids = customer_df['Customer_ID'].tolist()
            num_dummy_aml_trans = min(100, NUM_TRANSACTIONS // 100) # Generate a small number of dummy transactions
            for i in range(num_dummy_aml_trans):
                cust_id = random.choice(customer_ids)
                is_alert = 1 if random.random() < 0.2 else 0 # 20% chance of alert
                anomaly_score = random.uniform(-0.5, 0.0) if is_alert else random.uniform(0.0, 0.5)
                dummy_transactions.append({
                    'Transaction_ID': f'DUMMY_TRANS{i:04d}',
                    'Customer_ID': cust_id,
                    'Amount_USD': random.uniform(100, 100000),
                    'Anomaly_Score': anomaly_score,
                    'AML_Alert_Flag': 'ALERT' if is_alert else 'OK',
                    'Is_Suspicious_Label': is_alert # For consistency
                })
            aml_results_df = pd.DataFrame(dummy_transactions)


    # 3. Calculate Integrated Risk Scores
    integrated_risk_df = calculate_integrated_risk(customer_df.copy(), sanctions_results_df.copy(), aml_results_df.copy())

    # 4. Generate Consolidated Alerts Summary
    final_alerts_summary_df = generate_alerts_summary(integrated_risk_df.copy(), alert_threshold_score=50)

    # Save the final alerts summary
    final_alerts_summary_df.to_csv('final_aml_integrated_alerts.csv', index=False)
    print("\nFinal integrated AML alerts saved to 'final_aml_integrated_alerts.csv'")

    print("\n--- Integrated Risk Scoring and Alert Management Complete ---")


--- Starting Integrated Risk Scoring and Alert Management ---
Successfully loaded UK Sanctions List_mean.csv with 'latin1' encoding. Shape: (12376, 6)
FATAL: No suitable name column found in sanctions CSV. Using dummy data for sanctions.
An unexpected error occurred during sanctions data loading/cleaning: No name column found. Generating dummy sanctions data.
Generated fallback sanctions data due to error.
Successfully loaded customer data from customer_data.csv. Shape: (12820, 8)
Standardized customer column names: {'Customer_Id': 'Customer_ID', 'Customer_Name': 'Customer_Name', 'Customer_Address': 'Customer_Address', 'Customer_Dob': 'Customer_DOB', 'Customer_Nationality': 'Customer_Nationality', 'Customer_Country': 'Customer_Country', 'Customer_Industry': 'Customer_Industry', 'Onboarding_Date': 'Onboarding_Date'}
Customer DataFrame columns after standardization: ['Customer_ID', 'Customer_Name', 'Customer_Address', 'Customer_DOB', 'Customer_Nationality', 'Customer_Country', 'Customer_